In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
df_users = pd.read_csv('/kaggle/input/meta-kaggle/Users.csv')

In [ ]:
df_users.head()

In [ ]:
def scrape_data(df):
    for index in df.index:
        #time.sleep(1)
        row = df.iloc[index]
    
        username = row.UserName
        profile_url = '{}{}'.format(KAGGLE_BASE_URL, username)
        displayname = row.DisplayName

        result = requests.get(profile_url)
        src = result.content
        soup = BeautifulSoup(src, 'html.parser')
        soup = soup.find_all("div", id="site-body")[0].find("script")

        user_info = re.search('Kaggle.State.push\(({.*})', str(soup)).group(1)
        user_dict = json.loads(user_info)
    
        city = user_dict['city']
        region = user_dict['region']
        country = user_dict['country']
        avatar_url = user_dict['userAvatarUrl']
        occupation = user_dict['occupation']
        organization = user_dict['organization']
        github_user = user_dict['gitHubUserName']
        twitter_user = user_dict['twitterUserName']
        linkedin_url = user_dict['linkedInUrl']
        website_url = user_dict['websiteUrl']
        last_active = user_dict['userLastActive']
        num_followers = user_dict['followers']['count']
        num_following = user_dict['following']['count']

        num_posts = user_dict['discussionsSummary']['totalResults']
        num_datasets = user_dict['datasetsSummary']['totalResults']
        num_kernels = user_dict['scriptsSummary']['totalResults']
        num_comps = user_dict['competitionsSummary']['totalResults']


        df.loc[index, 'Image'] = '<a href="{}" target="_blank" title="{}"><img src="{}" width="100" height="100"></a>'.format(
            profile_url, displayname, avatar_url)
        df.loc[index, 'NumFollowers'] = num_followers
        df.loc[index, 'NumFollowing'] = num_following
        df.loc[index, 'NumPosts'] = num_posts
        df.loc[index, 'NumDatasets'] = num_datasets
        df.loc[index, 'NumKernels'] = num_kernels
        df.loc[index, 'NumCompetitions'] = num_comps
        df.loc[index, 'Country'] = country
    return df



In [ ]:
def display_html(df, cols=None, index=False, na_rep='', num_rows=0):
    if num_rows == 0:
        df_table = df.to_html(columns=cols, index=index, na_rep=na_rep, escape=False, render_links=True)
        display(HTML(df_table))
    else:
        df_table = df.head(num_rows).to_html(columns=cols, index=index, na_rep=na_rep, escape=False, render_links=True)
        display(HTML(df_table))
KAGGLE_BASE_URL = "https://www.kaggle.com/"
df_users['RegisterDate'] = pd.to_datetime(df_users['RegisterDate'],format='%m/%d/%Y')
df_users['Joined'] = df_users.RegisterDate.apply(lambda x: str(2020-x.year)+' years ago')
df_users.reset_index(drop=True,inplace=True)
df_users = scrape_data(df_users)
df_users.NumFollowers = df_users.NumFollowers.astype(int)
display_html(df_users, cols=['Image','UserName', 'DisplayName','Joined','NumFollowers','Country'])